In [18]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
from torchvision import transforms, datasets
import torchvision
import requests
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from torchvision import datasets, transforms, models
from torch.nn import init
import pandas as pd
from pyAudioAnalysis import audioBasicIO as io
from os import path
import os
from pydub import AudioSegment
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
import math

In [19]:
class AudioUtil():
  # ----------------------------
  # Load an audio file. Return the signal as a tensor and the sample rate
  # ----------------------------
  @staticmethod
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    return (sig, sr)

  @staticmethod
  def rechannel(aud, new_channel):
    sig, sr = aud

    if (sig.shape[0] == new_channel):
      # Nothing to do
      return aud

    if (new_channel == 1):
      # Convert from stereo to mono by selecting only the first channel
      resig = sig[:1, :]
    else:
      # Convert from mono to stereo by duplicating the first channel
      resig = torch.cat([sig, sig])

    return ((resig, sr))

  @staticmethod
  def resample(aud, newsr):
    sig, sr = aud

    if (sr == newsr):
      # Nothing to do
      return aud

    num_channels = sig.shape[0]
    # Resample first channel
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      # Resample the second channel and merge both channels
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))

  @staticmethod
  def pad_trunc(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms

    if (sig_len > max_len):
      # Truncate the signal to the given length
      sig = sig[:,:max_len]

    elif (sig_len < max_len):
      # Length of padding to add at the beginning and end of the signal
      pad_begin_len = random.randint(0, max_len - sig_len)
      pad_end_len = max_len - sig_len - pad_begin_len

      # Pad with 0s
      pad_begin = torch.zeros((num_rows, pad_begin_len))
      pad_end = torch.zeros((num_rows, pad_end_len))

      sig = torch.cat((pad_begin, sig, pad_end), 1)
      
    return (sig, sr)

  @staticmethod
  def time_shift(aud, shift_limit):
    sig,sr = aud
    _, sig_len = sig.shape
    shift_amt = int(random.random() * shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)

  @staticmethod
  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80

    # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
    spec = torchaudio.transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

    # Convert to decibels
    spec = torchaudio.transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

  @staticmethod
  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = torchaudio.transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = torchaudio.transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

    return aug_spec

In [20]:
class MultilabelClassifier(nn.Module):
    def __init__(self):
        super().__init__()
    
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization
        self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        # Second Convolution Block
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Second Convolution Block
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]
        
        self.conv = nn.Sequential(*conv_layers)
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.sig = nn.Sigmoid()
        self.lin = nn.Linear(in_features=64, out_features=7)

        
    def forward(self, x):

        x = self.conv(x)
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        x = self.lin(x)
#         x = self.lin2(x)
#         x = self.lin3(x)
        x = self.sig(x)
#         print(x.shape)
        
        return x

In [21]:
class LabelToPositivityModel(nn.Module):
    def __init__(self):
        super(LabelToPositivityModel, self).__init__()
        self.fc1 = nn.Linear(7, 512)
        self.relu1 = nn.ReLU()

    
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()


        self.fc3 = nn.Linear(256, 128)
        self.relu3 = nn.ReLU()


        self.fc4 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()
        
        self.fchehe1 = nn.Linear(7,1)



    def forward(self, x):
        x = self.fchehe1(x)
#         x = self.fchehe2(x)
#         x = self.fc1(x)
#         x = self.relu1(x)
        
#         x = self.fc2(x)
#         x = self.relu2(x)

#         x = self.fc3(x)
#         x = self.relu3(x)

#         x = self.fc4(x)
#         x = self.sigmoid(x)

        return x

In [22]:
import torch
import torch.nn as nn

class LabelToClusterModel(nn.Module):
    def __init__(self):
        super(LabelToClusterModel, self).__init__()
        self.fc1 = nn.Linear(7, 512)
        self.relu1 = nn.ReLU()
#         self.relu1 = nn.LeakyReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
#         self.relu2 = nn.LeakyReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, 128)
        self.relu3 = nn.ReLU()
#         self.relu3 = nn.LeakyReLU()
        self.dropout3 = nn.Dropout(0.5)
        self.fc4 = nn.Linear(128, 500)
        self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()
        
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)
        nn.init.xavier_uniform_(self.fc3.weight)
        nn.init.zeros_(self.fc3.bias)
        nn.init.xavier_uniform_(self.fc4.weight)
        nn.init.zeros_(self.fc4.bias)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
#         x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
#         x = self.dropout2(x)
        x = self.fc3(x)
        x = self.relu3(x)
#         x = self.dropout3(x)
        x = self.fc4(x)
#         x = self.softmax(x)
#         self.sigmoid(x)
        return x

In [23]:
def get_spotify_id(song_name, artist_name, token):

    headers = {
        "Authorization": "Bearer " + token,
        "Content-Type": "application/json"
    }

    query_params = {
        "q": f"{song_name} {artist_name}",
        "type": "track"
    }
    print(song_name)
    response = requests.get("https://api.spotify.com/v1/search", headers=headers, params=query_params)
    try:

        if response.json()["tracks"]["items"] is not None:
            results = response.json()["tracks"]["items"]
            output = []
            return results[0]["id"]

    except KeyError:
        print("Spotify error")
        return "Error"

In [54]:

def get_spotify_labels(id, token):
    labels = []
    r=requests.get("https://api.spotify.com/v1/audio-features?ids=" + id,
                   headers={"Accept": "application/json",
                            "Content-Type": "application/json",
                            "Authorization": "Bearer " + token})
    res = r.json()["audio_features"][0]
    labels.append(res["acousticness"] * 100)
    labels.append(res["danceability"] * 100)
    labels.append(res["energy"] * 100)
    labels.append(res["instrumentalness"] * 100)
    labels.append(res["liveness"] * 100)
    labels.append(res["speechiness"] * 100)
    labels.append(res["valence"] * 100)
    
    return labels



In [55]:
def getSongLabels(song_name, artist_name, token):
    id = get_spotify_id(song_name, artist_name, token)
    print(id)
    return get_spotify_labels(id, token)

In [56]:
def recommendMoviesFromLabels(labels, pathToModel, movieMatches, movies):
    
    model = torch.load(pathToModel)
    
    predictedCluster = "asd"
    model.eval()
    with torch.no_grad():
        inputt = (torch.Tensor(labels)).unsqueeze(0)

        predictedCluster = model(inputt)

        _, predictedCluster = torch.max(predictedCluster.data, 1)
        predictedCluster = predictedCluster.item()


    movieCluster = movieMatches[movieMatches["spotify_cluster_id"] == predictedCluster]["movie_cluster_id"].item()

    matches = movies[movies["cluster_id"] == movieCluster]
    
    final = []
    for movie in matches["movie"]:
        if movie not in final:
            final.append(movie)
    return final

In [57]:
def recMovieFromSpoti(pathToModel, song_name, artist_name, token, movieMatches, movies):
    songLabels = getSongLabels(song_name, artist_name, token)
    print("labels: ", songLabels)
    recs = recommendMoviesFromLabels(songLabels,pathToModel,movieMatches,movies )
    for movie in recs:
        print(movie)

In [58]:
def getSpectrogram(path):
    duration = 20000
    sr = 44100
    channel = 2
    shift_pct = 0.4
    sound = AudioSegment.from_mp3(path)
    sound.export("./testSong.wav", format="wav")
    aud = AudioUtil.open("./testSong.wav")

    reaud = AudioUtil.resample(aud, sr)
    rechan = AudioUtil.rechannel(reaud, channel)

    dur_aud = AudioUtil.pad_trunc(rechan, duration)
    shift_aud = AudioUtil.time_shift(dur_aud, shift_pct)
    sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    
    return aug_sgram

In [59]:
def getSpotifyLabelFromWave(spectro):
    model = MultilabelClassifier()
    model.load_state_dict(torch.load("./final.pt"))
    model.eval()
    with torch.no_grad():
        inputt = (torch.Tensor(spectro)).unsqueeze(0)
        result = model(inputt)
    return [i * 100 for i in result[0].tolist()]

In [60]:
def recMovieFromMp3(path="./testSong.mp3"):
    spectro = getSpectrogram(path)
    labels = getSpotifyLabelFromWave(spectro)
    recs = recommendMoviesFromLabels(labels,"./labeltocluster.pt" ,movieMatches,movies )
    for movie in recs:
        print(movie)

In [61]:
def getRecsFromPos(song_name, artist, token, path="./labeltopos.pt"):
    labels = [i/100 for i in getSongLabels(song_name, artist, token)]
    model = torch.load(path)
    model.eval()
    with torch.no_grad():
        inputt = torch.Tensor(labels).unsqueeze(0)
        result = model(inputt)
        
    result = result[0].item()
    
    closestIndex = 0
    closestDiff = 1000
    for i in range(movieRatings.shape[0]):
        diff = abs(movieRatings["overall_positiveness_score"][i] - result)
        if diff < closestDiff:
            closestDiff = diff
            closestIndex = i
#     print(closestIndex)
    if (closestIndex < 0):
        closestIndex += 3
    elif closestIndex > movieRatings.shape[0]-2:
        closestIndex -= 3

    recList = []
    recList.append(movieRatings["filename"][closestIndex-2])
    recList.append(movieRatings["filename"][closestIndex-1])
    recList.append(movieRatings["filename"][closestIndex])
    recList.append(movieRatings["filename"][closestIndex+1])
    recList.append(movieRatings["filename"][closestIndex+2])
    for rec in recList:
        print(rec)
     

In [62]:
movieMatches = pd.read_csv("./cluters_translation_table.csv", low_memory= False)
movies = pd.read_csv("./movie_soundtracks_table.csv", low_memory=False)
movieRatings = pd.read_csv("./sentiment_analysis_results.csv", low_memory=False)

In [50]:
token = "BQA5v7Nzp6uUB0hqMqKc6FB9fz0jjY1quCvl7vR2zvW6Q2UNo9Ul2bY7PCWiwkloR8JgPnXkID9O2uxWmD9vaR5ECbGABk2kvk8tnuxElLOJH165EAvqoDngYqG6wg7QklvW4fqaP-2S3s6l9Jq19KTzWv37DnkiG08ScZayWDsUIYZckpY0F5reT1ouVkEInQvA"

In [63]:







recMovieFromSpoti("./labeltocluster.pt", "ne me quitte pas", "jacques brel", token, movieMatches,movies )


ne me quitte pas
6IRA4KOVbtiGiTdYoEThJN
labels:  [82.1, 41.6, 23.599999999999998, 0.012, 18.9, 4.99, 30.2]
Casino
Mad Max: Fury Road
The Terminator
Game of Thrones


In [44]:
recMovieFromMp3("./volare.mp3")

Spider-Man: Into the Spider-Verse
3 Idiots
The Hunt
Casino
Fargo
Prisoners
La Haine


In [41]:
getRecsFromPos("ne me quitte pas", "jacques brel", token)

ne me quitte pas
6IRA4KOVbtiGiTdYoEThJN
file_1781.txt
file_2608.txt
file_1819.txt
file_2074.txt
file_827.txt
